In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, gc, warnings
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")

#import transformers
from transformers import pipeline

In [ ]:
df=pd.read_csv("latest_ticket_data.csv")

In [ ]:
classifier = pipeline("zero-shot-classification")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
candidate_labels = df['Category'].unique()

In [ ]:
LE = LabelEncoder()
df['Label'] = LE.fit_transform(df['Category'])
print(df)

                                            Description     Category  Label
0     hi since recruiter lead permission approve req...  Application      0
1     re expire days hi ask help update passwords co...  Application      0
2     verification warning hi has got attached pleas...  Application      0
3     please dear looks blacklisted receiving mails ...  Application      0
4     dear modules report report cost thank much reg...  Application      0
...                                                 ...          ...    ...
2995  sent tuesday feedback follow up conf call hell...     Database      1
2996  sent monday issues hi keep getting errors whil...     Database      1
2997  sent monday en working properly hi guys we hav...     Database      1
2998  sent wednesday july hi please log incident for...     Database      1
2999  sent tuesday july connection issues hello have...     Database      1

[3000 rows x 3 columns]


In [ ]:
# Generate samples that contains K samples of each class

def gen_sample(df, sample_size, num_classes):

    df_1 = df[(df["Label"]<num_classes + 1)].reset_index().drop(["index"], axis=1).reset_index().drop(["index"], axis=1)
    train = df_1[df_1["Label"] == np.unique(df_1['Label'])[0]].sample(sample_size)

    train_index = train.index.tolist()

    for i in range(1,num_classes):
        train_2 = df_1[df_1["Label"] == np.unique(df_1['Label'])[i]].sample(sample_size)
        train = pd.concat([train, train_2], axis=0)
        train_index.extend(train_2.index.tolist())

    test = df_1[~df_1.index.isin(train_index)]

    return train, test

In [ ]:
train_dataset, test_dataset = gen_sample(df, 10, 5)

In [ ]:
train_dataset.shape

(50, 3)

In [ ]:
train_dataset['pred_label'] = train_dataset["Description"].map(lambda x: classifier(x, candidate_labels)['scores'].index(max(classifier(x, candidate_labels)['scores'])))

In [ ]:
train_dataset['pred_label']

558     0
502     0
478     0
303     0
73      0
294     0
406     0
205     0
544     0
537     0
866     0
2971    0
693     0
826     0
2887    0
840     0
832     0
2748    0
2777    0
2735    0
1172    0
1023    0
1168    0
2637    0
2482    0
2421    0
2526    0
936     0
2560    0
2584    0
1993    0
2241    0
2104    0
2341    0
1819    0
2278    0
2350    0
2182    0
2115    0
2322    0
1247    0
1783    0
1359    0
1378    0
1299    0
1648    0
1416    0
1316    0
1556    0
1442    0
Name: pred_label, dtype: int64

In [ ]:
train_dataset['Label']

558     0
502     0
478     0
303     0
73      0
294     0
406     0
205     0
544     0
537     0
866     1
2971    1
693     1
826     1
2887    1
840     1
832     1
2748    1
2777    1
2735    1
1172    2
1023    2
1168    2
2637    2
2482    2
2421    2
2526    2
936     2
2560    2
2584    2
1993    3
2241    3
2104    3
2341    3
1819    3
2278    3
2350    3
2182    3
2115    3
2322    3
1247    4
1783    4
1359    4
1378    4
1299    4
1648    4
1416    4
1316    4
1556    4
1442    4
Name: Label, dtype: int64

In [ ]:
train_dataset

,Description,Category,Label,pred_label
558,accounts good morning please note changes deac...,Application,0,0
502,client dear please atom his updated credential...,Application,0,0
478,needed report dear colleagues kindly ask provi...,Application,0,0
303,phone hi phone charging tried everything boote...,Application,0,0
73,configuration periscope hello working componen...,Application,0,0
294,please help confluence si saved conversation c...,Application,0,0
406,marketplace hello due leaving hand over few br...,Application,0,0
205,hi forwarded element inside which thanks re ve...,Application,0,0
544,name change hi teams please name change regula...,Application,0,0
537,transfer hi please fill transfer form thanks k...,Application,0,0


In [ ]:
seq = train_dataset['Description'][1993]
print(classifier(seq, candidate_labels)['scores'])
print(max(classifier(seq, candidate_labels)['scores']))
print(classifier(seq, candidate_labels)['scores'].index(max(classifier(seq, candidate_labels)['scores'])))

[0.35130101442337036, 0.27008160948753357, 0.17222189903259277, 0.13554050028324127, 0.0708550214767456]
0.35130101442337036
0


In [ ]:
y_pred = train_dataset['pred_label'].values
y_true = train_dataset['Label'].values
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)

0.2